# Notebook to run gPhoton
Run gPhoton to create GALEX light curves etc.
Setup your source parameter and setting in the cell below.

In [1]:
region_name = "TDS" # "WD" #"MDIS_10-800" # _ELAISN1
srcs_ids = [1891 ,3403 ,16149 ,45461 ,] 
outdir = "./resources/gPhoton_out/"
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+"_cat.fits"

In [2]:
%matplotlib widget
import ipywidgets as widgets
import gPhoton
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii
from astropy.table import Table
from astropy import units as uu
import os
from vasca.utils import mag2flux, flux2mag, tgalex_to_astrotime
import vasca.visualization as vvis
from vasca.region import Region

rg = Region()
rg.load_from_fits(region_fname)

#Subselect sources based on choice
if len(srcs_ids)>0:
    rg.tt_sources.add_index("rg_src_id")
    idx_srcs = rg.tt_sources.loc_indices["rg_src_id", srcs_ids]
    tt_srcs = Table(rg.tt_sources[idx_srcs])
else:
    tt_srcs = rg.tt_sources

display(tt_srcs)

fd_src_id,nr_det,ra,dec,pos_err,pos_xv,pos_var,pos_cpval,pos_rchiq,coadd_src_id,coadd_dist,obs_filter_id,sel,flux,flux_err,flux_nxv,flux_var,flux_cpval,flux_rchiq,coadd_ffactor,coadd_fdiff_s2n,rg_fd_id,rg_src_id,nr_fd_srcs,hr,hr_err,otype,main_id,otypes,z_value,distance_distance,distance_unit,distance_result
,,deg,deg,arcsec,arcsec2,arcsec2,,,,arcsec,,,1e-06 Jy,1e-06 Jy,,1e-12 Jy2,,,,,,,,,,,,,,,,arcsec
int32,int32[2],float64,float64,float32,float32,float32,float32,float32,int64,float32,int32[2],bool,float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],float32[2],int64,int32,int32,float32,float32,bytes32,bytes32,bytes32,float64,float64,bytes4,float32
-1,35 .. 5,334.61900740472345,-0.0033813554025151345,0.08654309,-0.13358614,0.16820572,0.97304916,0.5630017,2410,0.18017368,1 .. 2,True,121.59981 .. 40.61505,0.32700354 .. 0.6793138,0.0012762442 .. 0.0012225002,22.828146 .. 9.024125,2.1874502e-27 .. 0.001186736,6.2001452 .. 4.5217357,1.0003184 .. 1.007664,0.08853569 .. 0.31262133,-1,1891,2,0.33295098,0.0059931264,WD*,PB 5130,WD*|WD*|WD*|WD*|*|Opt|UV|WD?|WD?,1.3299500,121.7450,pc,0.3261
-1,35 .. 4,333.5413675869932,0.8794347888579439,0.10153956,-0.2568612,0.15229508,0.99949604,0.37336546,6503,0.08153211,1 .. 2,True,20.24083 .. 5.189896,0.16559538 .. 0.40037686,0.06014167 .. -0.020882767,25.663895 .. 0.08260181,0.0 .. 0.9400651,26.422163 .. 0.13361952,0.9598218 .. 1.0394988,-3.864632 .. 0.3762283,-1,3403,2,0.34424755,0.02826839,WD*,GALEX J221409.8+005245,WD*|WD*|WD*|*|Opt|UV,--,439.0000,pc,0.2825
-1,28 .. 9,150.65361426058803,2.643111670533707,0.08557914,-0.08412055,0.19236396,0.8580123,0.66507405,44620,0.12717035,1 .. 2,True,106.96241 .. 66.50696,0.34322292 .. 0.6652977,0.0007799396 .. 0.0010192677,12.584814 .. 8.828741,5.945311e-11 .. 0.026203794,3.846406 .. 2.1749918,1.0030466 .. 1.0014558,0.71483445 .. 0.09157375,-1,16149,2,0.6084019,0.006708973,WD?,GALEX J100236.8+023834,*|*|G|Opt|UV|WD?|WD?,1.9900000,293.2980,pc,0.4992
-1,12 .. -1,244.14364033210146,54.16991388586225,0.16000032,-0.1349428,0.17470829,0.4975613,0.578193,12469,0.3236669,1 .. 0,True,46.202312 .. -1.0,0.4356251 .. -1.0,0.019377798 .. -100.0,43.763363 .. -1.0,7.33e-43 .. -1.0,20.80549 .. -1.0,1.0119411 .. -100.0,1.0512818 .. -10000.0,-1,45461,1,-1.0,-1.0,WD?,GALEX J161634.4+541011,*|*|Opt|UV|WD?|WD?,--,93.6920,pc,0.9987


### Do aperture photometry for each visit with gPhoton

In [3]:
#print(dd_gph["gAperture"].keys())
#print("\n", dd_gph["gFind"]["NUV"].keys())
#print(dd_gph["gAperture"]['photons'])
def plot_lc(rg_src_id):

    #Get table entries for that source
    tt_src = tt_srcs[tt_srcs["rg_src_id"]==rg_src_id]

    #Load file
    outfile_app = outdir+"gPhoton_"+str(round(tt_src["ra"][0],5))+"_"+str(round(tt_src["dec"][0],5))+"_app.npy"
    dd_gph = np.load(outfile_app,allow_pickle='TRUE').item()

    # Get gphoton lc
    keep_keys = ('t_mean', 'exptime','flux_bgsub', 'flux_bgsub_err','mag_bgsub','mag_bgsub_err_2',"flags","mag_mcatbgsub","mag_mcatbgsub_err_2", "mag")
    dd_gap = {x: dd_gph["gAperture"][x] for x in keep_keys if x in dd_gph["gAperture"]}
    dd_gap["s2n"]=dd_gap['flux_bgsub']/dd_gap['flux_bgsub_err']
    
    # Units of flux_bgsub are in erg sec^-1 cm^-2 Å^-1. . Get also Jy flux from AB magnitude
    dd_gap['flux'],dd_gap['flux_err'] = mag2flux(dd_gap["mag_mcatbgsub"],dd_gap["mag_mcatbgsub_err_2"])
    dd_gap['t_mean']= tgalex_to_astrotime(dd_gap['t_mean'],"mjd")
    # Units of time are in "GALEX Time" = "UNIX Time" - 315964800, change to MJD
    
    tt_gap = Table(data= dd_gap)
    sel_gap = tt_gap["s2n"]>3.5

    fig_lc_src = plt.figure("Light curve",figsize=(8, 4), clear=True)
    vvis.plot_light_curve(rg, rg_src_ids = rg_src_id,fig=fig_lc_src, flux_var="flux") #

    plt.errorbar(tt_gap[sel_gap]['t_mean'],1.132*tt_gap[sel_gap]['flux'],yerr=tt_gap[sel_gap]['flux_err'],fmt=".")
    plt.yscale('log')
    plt.show()

    display(tt_src)
    display(tt_gap[sel_gap])
widgets.interactive( plot_lc, rg_src_id=tt_srcs["rg_src_id"])

interactive(children=(Dropdown(description='rg_src_id', options=(1891, 3403, 16149, 45461), value=1891), Outpu…

In [4]:
# Calculate apperture flux correctin from http://www.galex.caltech.edu/researcher/techdoc-ch5.html
f1 = mag2flux(19)
f2 = mag2flux(19-0.20)
mag = flux2mag(f2)
print(f1,f2,f2/f1, mag)


91.20108393559062 1e-06 Jy 109.64781961431808 1e-06 Jy 1.202264434617413 18.8 mag(AB)


In [5]:
1/(60*60)

0.0002777777777777778